In [239]:
import torch.nn.functional as F
from torch.autograd import Variable
import copy
from collections import Counter
import torch 
import numpy as np
import torch.nn as nn

In [240]:
def read_data(path):
    f = open(path, "r")
    lines = f.readlines()
    f.close()
    data = []
    for l in lines:
        labelSplit = l.replace('\n','').split(' ', 1)
        data.append([labelSplit[0], [word.lower() for word in labelSplit[1].split()]])
    return data

data = read_data('./questions.txt')

In [241]:
def remove_stop_words(data, path):
    stop_words = []
    with open(path) as f:
        stop_words = [word for line in f for word in line.split(",")]
    data_without_stop_words = []
    for k, v in data:
        words = [t for t in v if t not in stop_words]
        data_without_stop_words.append((k, words))
    return data_without_stop_words

data = remove_stop_words(data, './stop_words.txt')

In [242]:
def get_labels(data):
    _labels = []
    for k,v in data:
        _labels.append(k)   
    _unique_label = list(set(_labels))
    _unique_label_dict = {}
    for k,v in enumerate(_unique_label):
        _unique_label_dict[v] = k
    return _unique_label_dict

labels = get_labels(data)

In [243]:
def append_labels(data, labels):        
    cleaned_data = []
    for k,v in data:
        cleaned_data.append((labels[k],v))
        
    return np.array(cleaned_data)

data = append_labels(data, labels)

In [257]:
def create_indexed_vocab(data):
    vocab = []
    for _, sent in data:
        for word in sent:
            vocab.append(word)
    count = Counter(vocab)
    count = {w : count[w] for w in count if count[w] >= 2}
    vocab = []
    for k, v in count.items():
        vocab.append(k)
    indexed_vocab = {word: idx for idx, word in enumerate(vocab)}
    return indexed_vocab
def create_vocab(data):
    total_words_orig = []
    for k,sent in data:
        for word in sent:
            total_words_orig.append(word)
    total_words = list(set(total_words_orig))
    total_words_str = ' '.join(total_words)
    vocab = set(total_words_str.split()) 
    word2idx = {word: idx for idx, word in enumerate(vocab)} # create word index
    return word2idx
word2idx=create_vocab(data)
word2idx['#UNK#'] = len(word2idx)
indexed_vocab = create_indexed_vocab(data)
indexed_vocab['#UNK#'] = len(indexed_vocab)

In [258]:
def load_glove_embeddings(path, indexed_vocab, embedding_dim=300):
    with open(path) as f:
        embeddings = np.zeros((len(indexed_vocab), embedding_dim))
        for line in f.readlines():
            values = line.split()
            word = values[0]
            index = indexed_vocab.get(word)
            if index:
                vector = np.array(values[1:], dtype='float32')
                embeddings[index] = vector
        return torch.from_numpy(embeddings).float()

glove_random = load_glove_embeddings('./glove.txt', indexed_vocab)
glove_pre = load_glove_embeddings('./glove.txt', word2idx)
embeddings_random = nn.Embedding(glove_random.size(0), glove_random.size(1))
embeddings_pretrained = nn.Embedding.from_pretrained(glove_pre, freeze=False)

In [259]:
def split_train_test(data, test_ratio):
    data_copy = copy.deepcopy(data)
    np.random.shuffle(data_copy)
    test_set_size = int(len(data) * test_ratio)
    test = data_copy[:test_set_size]
    train = data_copy[test_set_size:]
    return train, test

train, test = split_train_test(data, 0.2)

In [260]:
class BOWClassifier(nn.Module):
    def __init__(self,input_size,hidden_size, num_labels):
        super(BOWClassifier, self).__init__()
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size,num_labels)
        #self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        #output = self.sigmoid(output)
        return F.softmax(output)

In [264]:
input_size = 300
hidden_size = 10
num_labels = 51
bow = BOWClassifier(input_size, hidden_size, num_labels)

In [265]:
def make_bow_vector(sentence, indexed_vocab):
    pt_tensor= torch.zeros(300, dtype=torch.long)
    count = 0
    for word in sentence:
        count += 1
        if word not in indexed_vocab:
            pt_tensor = torch.add(pt_tensor, embeddings_pretrained(torch.LongTensor([indexed_vocab['#UNK#']]))[0])
        else:
            pt_tensor = torch.add(pt_tensor, embeddings_pretrained(torch.LongTensor([indexed_vocab[word]]))[0])
    pt_tensor=torch.div(pt_tensor, count)
    return pt_tensor

def get_bow_rep(data):
    bow_data = []
    for label, sent in data:
        bow_data.append(make_bow_vector(sent, word2idx))
    return torch.stack(bow_data)
        
training_set = get_bow_rep(train)

In [266]:
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [ ]:
#training
n_epochs = 2
batch_size = 100
for epoch in range(n_epochs):
    permutation = torch.randperm(training_set.size()[0])
    for i in range(0, training_set.size()[0], batch_size):
        opt.zero_grad()
        indices = permutation[i:i + batch_size]
        batch_features = training_set[indices]
        batch_labels = torch.LongTensor([label for label, sent in train[indices]])
        batch_outputs = bow(batch_features)
        loss = loss_function(batch_outputs, batch_labels)
        print(loss)
        loss.backward(retain_graph=True)
        opt.step()
        print("done")
    

C:\Users\joshy\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


tensor(-0.0194, grad_fn=<NllLossBackward>)


In [ ]:
# cdata=[]
# target = 
# for l,d in b:
#     bow_vec = make_bow_vector(d, indexed_vocab)
#     cdata.append(list(bow_vec.tolist()))
#     target.append(labels[l])
# c_tensor= torch.FloatTensor(cdata)
# t_tensor=torch.LongTensor(target)
# y_pred = bow(c_tensor)
# loss = loss_function(y_pred,t_tensor)
# loss.backward()
# opt.step()

In [152]:
#testing
test_len =len(test)
correct=0
for label,data in test:
    bow_vec = make_bow_vector(data, word2idx)
    logprobs = bow(bow_vec)
    print(logprobs)
    pred = np.argmax(logprobs.data.numpy())
    if pred==label:
        correct+=1
    print('prediction: {}'.format(pred))
    print('actual: {}'.format(label))
accuracy = correct/test_len
print('accuracy: {}'.format(accuracy))

C:\Users\joshy\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


tensor([0.0233, 0.0192, 0.0210, 0.0159, 0.0175, 0.0213, 0.0190, 0.0150, 0.0227,
        0.0207, 0.0234, 0.0164, 0.0203, 0.0235, 0.0147, 0.0144, 0.0183, 0.0151,
        0.0182, 0.0146, 0.0212, 0.0229, 0.0243, 0.0186, 0.0213, 0.0160, 0.0181,
        0.0246, 0.0145, 0.0253, 0.0160, 0.0195, 0.0226, 0.0249, 0.0238, 0.0186,
        0.0239, 0.0262, 0.0244, 0.0140, 0.0228, 0.0147, 0.0246, 0.0199, 0.0148,
        0.0155, 0.0215, 0.0185, 0.0165, 0.0210, 0.0152],
       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 18
tensor([0.0244, 0.0197, 0.0209, 0.0156, 0.0182, 0.0211, 0.0189, 0.0141, 0.0228,
        0.0207, 0.0239, 0.0164, 0.0202, 0.0246, 0.0144, 0.0145, 0.0182, 0.0146,
        0.0181, 0.0137, 0.0204, 0.0227, 0.0241, 0.0193, 0.0216, 0.0158, 0.0173,
        0.0233, 0.0143, 0.0260, 0.0159, 0.0194, 0.0220, 0.0246, 0.0238, 0.0184,
        0.0240, 0.0276, 0.0259, 0.0139, 0.0231, 0.0142, 0.0244, 0.0198, 0.0146,
        0.0148, 0.0212, 0.0192, 0.0170, 0.0208, 0.0155],
       grad_fn=<SoftmaxBac

       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 13
tensor([0.0236, 0.0189, 0.0213, 0.0157, 0.0178, 0.0208, 0.0189, 0.0148, 0.0226,
        0.0210, 0.0230, 0.0165, 0.0201, 0.0242, 0.0147, 0.0146, 0.0182, 0.0151,
        0.0188, 0.0143, 0.0209, 0.0228, 0.0242, 0.0187, 0.0216, 0.0159, 0.0176,
        0.0235, 0.0145, 0.0252, 0.0158, 0.0196, 0.0222, 0.0254, 0.0246, 0.0188,
        0.0231, 0.0272, 0.0253, 0.0136, 0.0234, 0.0148, 0.0243, 0.0198, 0.0147,
        0.0156, 0.0210, 0.0190, 0.0165, 0.0201, 0.0156],
       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 18
tensor([0.0230, 0.0207, 0.0202, 0.0162, 0.0179, 0.0214, 0.0186, 0.0140, 0.0235,
        0.0204, 0.0241, 0.0164, 0.0209, 0.0230, 0.0149, 0.0142, 0.0184, 0.0144,
        0.0172, 0.0141, 0.0214, 0.0226, 0.0243, 0.0188, 0.0207, 0.0163, 0.0184,
        0.0251, 0.0143, 0.0262, 0.0163, 0.0197, 0.0223, 0.0243, 0.0233, 0.0179,
        0.0246, 0.0258, 0.0252, 0.0141, 0.0227, 0.0145, 0.0244, 0.0202, 0.0147,
        0.0144, 0.0213,

       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 11
tensor([0.0242, 0.0191, 0.0210, 0.0157, 0.0177, 0.0219, 0.0186, 0.0150, 0.0235,
        0.0206, 0.0231, 0.0158, 0.0199, 0.0234, 0.0145, 0.0140, 0.0179, 0.0153,
        0.0186, 0.0147, 0.0214, 0.0225, 0.0233, 0.0185, 0.0208, 0.0159, 0.0181,
        0.0248, 0.0147, 0.0257, 0.0157, 0.0202, 0.0221, 0.0248, 0.0239, 0.0181,
        0.0234, 0.0266, 0.0254, 0.0143, 0.0220, 0.0140, 0.0253, 0.0201, 0.0147,
        0.0151, 0.0217, 0.0190, 0.0171, 0.0212, 0.0152],
       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 44
tensor([0.0236, 0.0211, 0.0200, 0.0154, 0.0187, 0.0216, 0.0180, 0.0135, 0.0254,
        0.0209, 0.0238, 0.0153, 0.0216, 0.0234, 0.0143, 0.0139, 0.0179, 0.0145,
        0.0169, 0.0137, 0.0212, 0.0225, 0.0241, 0.0190, 0.0210, 0.0157, 0.0184,
        0.0247, 0.0140, 0.0267, 0.0153, 0.0210, 0.0217, 0.0245, 0.0228, 0.0170,
        0.0238, 0.0266, 0.0275, 0.0140, 0.0222, 0.0133, 0.0238, 0.0208, 0.0145,
        0.0136, 0.0220,

       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 11
tensor([0.0242, 0.0201, 0.0204, 0.0161, 0.0182, 0.0214, 0.0192, 0.0139, 0.0230,
        0.0201, 0.0245, 0.0167, 0.0206, 0.0236, 0.0150, 0.0143, 0.0185, 0.0145,
        0.0173, 0.0136, 0.0211, 0.0231, 0.0231, 0.0194, 0.0205, 0.0157, 0.0172,
        0.0247, 0.0148, 0.0268, 0.0166, 0.0196, 0.0217, 0.0241, 0.0236, 0.0177,
        0.0245, 0.0265, 0.0249, 0.0142, 0.0232, 0.0144, 0.0251, 0.0203, 0.0142,
        0.0145, 0.0210, 0.0188, 0.0167, 0.0212, 0.0157],
       grad_fn=<SoftmaxBackward>)
prediction: 29
actual: 19
tensor([0.0240, 0.0195, 0.0211, 0.0155, 0.0182, 0.0208, 0.0193, 0.0141, 0.0228,
        0.0207, 0.0236, 0.0165, 0.0201, 0.0244, 0.0151, 0.0144, 0.0183, 0.0145,
        0.0187, 0.0136, 0.0212, 0.0227, 0.0230, 0.0194, 0.0209, 0.0158, 0.0170,
        0.0231, 0.0149, 0.0262, 0.0163, 0.0199, 0.0213, 0.0251, 0.0247, 0.0183,
        0.0234, 0.0279, 0.0260, 0.0138, 0.0239, 0.0145, 0.0244, 0.0198, 0.0141,
        0.0150, 0.0205,

...

       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 36
tensor([0.0235, 0.0198, 0.0208, 0.0159, 0.0182, 0.0214, 0.0193, 0.0143, 0.0230,
        0.0203, 0.0240, 0.0166, 0.0209, 0.0239, 0.0150, 0.0144, 0.0182, 0.0146,
        0.0179, 0.0142, 0.0212, 0.0233, 0.0232, 0.0192, 0.0210, 0.0156, 0.0176,
        0.0243, 0.0147, 0.0264, 0.0162, 0.0200, 0.0218, 0.0243, 0.0237, 0.0179,
        0.0240, 0.0268, 0.0250, 0.0140, 0.0233, 0.0145, 0.0244, 0.0201, 0.0141,
        0.0146, 0.0210, 0.0188, 0.0164, 0.0207, 0.0157],
       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 18
tensor([0.0232, 0.0196, 0.0211, 0.0155, 0.0178, 0.0216, 0.0195, 0.0148, 0.0231,
        0.0206, 0.0234, 0.0159, 0.0202, 0.0236, 0.0150, 0.0140, 0.0182, 0.0147,
        0.0188, 0.0148, 0.0220, 0.0227, 0.0229, 0.0191, 0.0208, 0.0159, 0.0182,
        0.0241, 0.0148, 0.0260, 0.0162, 0.0204, 0.0218, 0.0248, 0.0239, 0.0182,
        0.0239, 0.0269, 0.0251, 0.0143, 0.0228, 0.0141, 0.0244, 0.0196, 0.0140,
        0.0151, 0.0212,

       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 42
tensor([0.0242, 0.0198, 0.0207, 0.0161, 0.0181, 0.0214, 0.0188, 0.0142, 0.0229,
        0.0203, 0.0241, 0.0167, 0.0205, 0.0241, 0.0145, 0.0146, 0.0182, 0.0147,
        0.0176, 0.0139, 0.0204, 0.0230, 0.0242, 0.0191, 0.0214, 0.0158, 0.0175,
        0.0242, 0.0143, 0.0261, 0.0160, 0.0193, 0.0223, 0.0242, 0.0235, 0.0182,
        0.0243, 0.0268, 0.0252, 0.0140, 0.0228, 0.0144, 0.0248, 0.0201, 0.0148,
        0.0147, 0.0213, 0.0187, 0.0169, 0.0213, 0.0153],
       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 19
tensor([0.0232, 0.0197, 0.0208, 0.0158, 0.0175, 0.0215, 0.0194, 0.0148, 0.0228,
        0.0206, 0.0237, 0.0161, 0.0203, 0.0232, 0.0150, 0.0141, 0.0185, 0.0148,
        0.0182, 0.0148, 0.0218, 0.0227, 0.0236, 0.0190, 0.0207, 0.0161, 0.0182,
        0.0247, 0.0147, 0.0257, 0.0163, 0.0198, 0.0223, 0.0247, 0.0235, 0.0184,
        0.0244, 0.0261, 0.0243, 0.0144, 0.0226, 0.0143, 0.0246, 0.0198, 0.0144,
        0.0153, 0.0215,

       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 29
tensor([0.0239, 0.0193, 0.0208, 0.0161, 0.0180, 0.0211, 0.0188, 0.0145, 0.0228,
        0.0205, 0.0237, 0.0168, 0.0205, 0.0239, 0.0147, 0.0146, 0.0182, 0.0150,
        0.0178, 0.0140, 0.0206, 0.0231, 0.0242, 0.0187, 0.0214, 0.0157, 0.0175,
        0.0244, 0.0145, 0.0258, 0.0160, 0.0193, 0.0223, 0.0247, 0.0239, 0.0183,
        0.0238, 0.0265, 0.0249, 0.0138, 0.0231, 0.0148, 0.0248, 0.0202, 0.0149,
        0.0151, 0.0212, 0.0187, 0.0167, 0.0209, 0.0154],
       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 18
tensor([0.0220, 0.0193, 0.0212, 0.0157, 0.0180, 0.0209, 0.0185, 0.0148, 0.0237,
        0.0212, 0.0226, 0.0163, 0.0216, 0.0238, 0.0148, 0.0146, 0.0177, 0.0151,
        0.0184, 0.0149, 0.0212, 0.0234, 0.0247, 0.0182, 0.0219, 0.0157, 0.0186,
        0.0241, 0.0142, 0.0251, 0.0154, 0.0206, 0.0223, 0.0254, 0.0242, 0.0183,
        0.0226, 0.0265, 0.0257, 0.0132, 0.0235, 0.0150, 0.0232, 0.0201, 0.0147,
        0.0150, 0.0211,

       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 18
tensor([0.0235, 0.0199, 0.0211, 0.0154, 0.0183, 0.0213, 0.0195, 0.0143, 0.0230,
        0.0207, 0.0238, 0.0161, 0.0207, 0.0244, 0.0148, 0.0143, 0.0182, 0.0144,
        0.0186, 0.0143, 0.0213, 0.0229, 0.0231, 0.0196, 0.0212, 0.0156, 0.0177,
        0.0232, 0.0147, 0.0262, 0.0159, 0.0204, 0.0215, 0.0244, 0.0238, 0.0182,
        0.0238, 0.0277, 0.0259, 0.0141, 0.0232, 0.0140, 0.0238, 0.0196, 0.0138,
        0.0147, 0.0211, 0.0192, 0.0164, 0.0204, 0.0159],
       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 38
tensor([0.0239, 0.0197, 0.0209, 0.0159, 0.0180, 0.0213, 0.0187, 0.0144, 0.0229,
        0.0206, 0.0237, 0.0165, 0.0205, 0.0243, 0.0144, 0.0146, 0.0180, 0.0148,
        0.0180, 0.0141, 0.0204, 0.0229, 0.0245, 0.0189, 0.0217, 0.0158, 0.0178,
        0.0239, 0.0142, 0.0258, 0.0158, 0.0194, 0.0224, 0.0245, 0.0236, 0.0184,
        0.0240, 0.0270, 0.0254, 0.0139, 0.0228, 0.0145, 0.0244, 0.0199, 0.0149,
        0.0149, 0.0213,

       grad_fn=<SoftmaxBackward>)
prediction: 29
actual: 16
tensor([0.0243, 0.0189, 0.0213, 0.0160, 0.0178, 0.0208, 0.0183, 0.0147, 0.0223,
        0.0207, 0.0230, 0.0168, 0.0198, 0.0244, 0.0143, 0.0149, 0.0180, 0.0151,
        0.0188, 0.0139, 0.0201, 0.0225, 0.0250, 0.0184, 0.0219, 0.0160, 0.0174,
        0.0235, 0.0142, 0.0250, 0.0156, 0.0190, 0.0225, 0.0250, 0.0248, 0.0191,
        0.0231, 0.0272, 0.0257, 0.0136, 0.0230, 0.0149, 0.0248, 0.0198, 0.0153,
        0.0155, 0.0210, 0.0190, 0.0169, 0.0204, 0.0156],
       grad_fn=<SoftmaxBackward>)
prediction: 37
actual: 3
accuracy: 0.03669724770642202


In [ ]:
torch.cuda.is_available()